In [16]:
import pandas as pd
from sodapy import Socrata
from datetime import datetime
import pickle
from geopy import distance
import numpy as np
import time

In [17]:
start = time.time()

dict_dfs={}

dict_dfs={"grad":["nb39-jx2v"], "trees":["uvpi-gqnh"],"crashes":["h9gi-nx95"],"pc":["cuzb-dmcd"],
          "shootings":["833y-fsy8"], "arrests":["8h9b-rp9u"],"programs":["mbd7-jfnc"],"bins":["sxx4-xhzg"]}
for key in dict_dfs:
    x=dict_dfs[key][0]
    dict_dfs[key]={"key":x,"last_updated":0,"data":""}
    
dict_dfs

{'grad': {'key': 'nb39-jx2v', 'last_updated': 0, 'data': ''},
 'trees': {'key': 'uvpi-gqnh', 'last_updated': 0, 'data': ''},
 'crashes': {'key': 'h9gi-nx95', 'last_updated': 0, 'data': ''},
 'pc': {'key': 'cuzb-dmcd', 'last_updated': 0, 'data': ''},
 'shootings': {'key': '833y-fsy8', 'last_updated': 0, 'data': ''},
 'arrests': {'key': '8h9b-rp9u', 'last_updated': 0, 'data': ''},
 'programs': {'key': 'mbd7-jfnc', 'last_updated': 0, 'data': ''},
 'bins': {'key': 'sxx4-xhzg', 'last_updated': 0, 'data': ''}}

In [18]:
def get_data(data_dict):
    data_url = "data.cityofnewyork.us"
    token = "HnDwBjZUCabVQojpbl6aIDrDI"
    client = Socrata(data_url,token, timeout=120)
    for key in data_dict:
        data_set=data_dict[key]["key"]
        meta = client.get_metadata(data_set)
        if meta["viewLastModified"]>data_dict[key]["last_updated"]:
            data_dict[key]["last_updated"]=meta["viewLastModified"]
            data_dict[key]["data"]=client.get(data_set, limit=10000)
            lastUpdated=True
        print(key + " done")      
    
     

get_data(dict_dfs)


grad done
trees done
crashes done
pc done
shootings done
arrests done
programs done
bins done


In [19]:
def get_avgGradRate(df_school, df_scatter):
    df=df_school
    df1=df_scatter
    df["total_grads_of_cohort"]=df["total_grads_of_cohort"].apply(float)
    x=df.groupby(["dbn"]).mean()
    x=x.reset_index()
    x=x.rename(columns={"total_grads_of_cohort":"avg_perc Grads"})
    x=x[["dbn","avg_perc Grads"]]
    df1=df1.merge(x, on= "dbn", how="left")
    return df1



In [20]:
def get_dataframes(data_dict):
    for key in data_dict:
        if key in ["addresses","scatter"]:
            continue
        df=pd.DataFrame(data_dict[key]["data"])
        columns=df.columns
        if "latitude" in df.columns:
            df["location"]=list(zip(df["latitude"].to_list(),df["longitude"].to_list()))
            df=df[["location","latitude","longitude"]]
            df=df[df.location.notna()]
        if "lat" in df.columns:
            df["location"]=list(zip(df["lat"].to_list(),df["lon"].to_list()))
            df=df[["location","lat","lon"]]
            df=df.rename(columns={"lat":"latitude","lon":"longitude"})
            df=df[df.location.notna()]
        data_dict[key]["df"]=df
        print(key + " done")
    df=pd.read_csv("datasets/2016_DOE_High_School_Directory-2.csv")
    df["location"]=list(zip(df["Latitude"].to_list(),df["Longitude"].to_list()))
    df=df.rename(columns={"Latitude":"latitude","Longitude":"longitude"})
    df=df[["dbn","school_name","borough","location","latitude","longitude"]]
    data_dict["grad"]["df"]=data_dict["grad"]["df"].merge(df,on="dbn",how="left")
    df=get_avgGradRate(data_dict["grad"]["df"], df)
    data_dict["scatter"]={"df":df}
get_dataframes(dict_dfs)

grad done
trees done
crashes done
pc done
shootings done
arrests done
programs done
bins done


In [21]:
dict_dfs["grad"]["df"]

,dbn,school_name_x,demographic_category,demographic_variable,cohort_year,cohort,total_cohort,total_grads,total_grads_of_cohort,total_regents,...,local_of_grads,still_enrolled,still_enrolled_of_cohort,dropped_out,dropped_out_of_cohort,school_name_y,borough,location,latitude,longitude
0,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,2013,4 year August,36,25,69.4,23,...,8.0,3,8.3,7,19.4,Henry Street School for International Studies,Manhattan,"(40.713684, -73.986336)",40.713684,-73.986336
1,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,2012,4 year August,44,24,54.5,20,...,16.7,10,22.7,10,22.7,Henry Street School for International Studies,Manhattan,"(40.713684, -73.986336)",40.713684,-73.986336
2,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,2011,4 year August,73,46,63.0,41,...,10.9,18,24.7,7,9.6,Henry Street School for International Studies,Manhattan,"(40.713684, -73.986336)",40.713684,-73.986336
3,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,2010,4 year August,61,26,42.6,26,...,0.0,18,29.5,17,27.9,Henry Street School for International Studies,Manhattan,"(40.713684, -73.986336)",40.713684,-73.986336
4,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,2009,4 year August,85,49,57.6,44,...,10.2,28,32.9,8,9.4,Henry Street School for International Studies,Manhattan,"(40.713684, -73.986336)",40.713684,-73.986336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,11X275,HIGH SCHOOL OF COMPUTERS AND TECHNOLOGY,All Students,All Students,2006,5 year June,92,80,87.0,60,...,25.0,5,5.4,6,6.5,High School of Computers and Technology,Bronx,"(40.875953, -73.86197)",40.875953,-73.861970
9996,11X275,HIGH SCHOOL OF COMPUTERS AND TECHNOLOGY,All Students,All Students,2005,5 year June,94,79,84.0,55,...,30.4,10,10.6,4,4.3,High School of Computers and Technology,Bronx,"(40.875953, -73.86197)",40.875953,-73.861970
9997,11X275,HIGH SCHOOL OF COMPUTERS AND TECHNOLOGY,All Students,All Students,2004,5 year June,84,80,95.2,48,...,40.0,1,1.2,3,3.6,High School of Computers and Technology,Bronx,"(40.875953, -73.86197)",40.875953,-73.861970
9998,11X275,HIGH SCHOOL OF COMPUTERS AND TECHNOLOGY,All Students,All Students,2003,5 year June,3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,High School of Computers and Technology,Bronx,"(40.875953, -73.86197)",40.875953,-73.861970


In [22]:
def prep_grad(data_dict):
    df_school=data_dict["grad"]["df"]
    df_school=df_school[df_school.demographic_category=="All Students"]
    data_dict["grad"]["df"]=df_school
    
prep_grad(dict_dfs)


In [23]:
def get_distance(loc1,loc2):
    try:
        return distance.distance(loc1,loc2).meters
    except:
        return np.nan
    

def get_numFeat(dict_dfs):
    dbns=dict_dfs["grad"]["df"]["dbn"].to_list()
    df_school=dict_dfs["grad"]["df"]
    df_school=df_school[df_school.location.notnull()]
    dbns=set(df_school["dbn"].to_list())
    dict_dbn={}
    dict_keys={}
    dict_feat={}
    dict_dfs["scatter"]["feat"]=""
    
    for key in dict_dfs:
        if key != "scatter":
            dict_dbn={}
            print( "\n\n\n",key)
            if key != "grad":
                df=dict_dfs[key]["df"]
                df=df[df.location.notnull()]
                for dbn in dbns:
                    locSchool=df_school.loc[df_school["dbn"]==dbn,"location"].to_list()[0]
                    df["distance"]=df["location"].apply(lambda x: get_distance(locSchool,x))
                    numFeat=(df["distance"].values<500).sum()
                    dict_dbn[dbn]=numFeat
                dict_keys[key]=dict_dbn
    #dict_feat["feat"]=dict_keys
    dict_dfs["scatter"]["feat"]=dict_keys
                   
                    
            
 
    



get_numFeat(dict_dfs)

end=time.time()

print("/n/n",(end-start)/59.992)




 grad



 trees



 crashes



 pc



 shootings



 arrests



 programs



 bins
/n/n 17.43409232721915


In [24]:
def get_scatterDf(data_dict):
    dfScatter=data_dict["scatter"]["df"]
    #dfSchool=data_dict["grad"]["df"][["dbn", "demographic_category", "cohort_year", "total_grads_of_cohort"]]
    
    dfFeat=pd.DataFrame.from_dict(data_dict["scatter"]["feat"])
    dfFeat.reset_index(inplace=True)
    dfFeat=dfFeat.rename(columns={"index":"dbn"})
    dfScatter=dfScatter.merge(dfFeat,on="dbn",how="left")
    #dfScatter=dfScatter.merge(dfSchool,on="dbn",how="left")
    return dfScatter



x=get_scatterDf(dict_dfs)
x
#x.to_csv("datasets/dataForScatter.csv")

,dbn,school_name,borough,location,latitude,longitude,avg_perc Grads,trees,crashes,pc,shootings,arrests,programs,bins
0,01M292,Henry Street School for International Studies,Manhattan,"(40.713684, -73.986336)",40.713684,-73.986336,60.276087,3.0,21.0,5.0,14.0,73.0,17.0,1.0
1,01M448,University Neighborhood High School,Manhattan,"(40.712399, -73.984497)",40.712399,-73.984497,76.075000,3.0,15.0,5.0,13.0,54.0,12.0,0.0
2,01M450,East Side Community School,Manhattan,"(40.729589, -73.982555)",40.729589,-73.982555,79.267308,28.0,18.0,4.0,6.0,15.0,2.0,3.0
3,01M509,Marta Valle High School,Manhattan,"(40.720581, -73.985645)",40.720581,-73.985645,57.250877,55.0,27.0,3.0,22.0,44.0,13.0,2.0
4,01M539,"New Explorations into Science, Technology and ...",Manhattan,"(40.718895, -73.979308)",40.718895,-73.979308,97.886538,28.0,17.0,5.0,28.0,32.0,4.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,32K545,EBC High School for Public Service - Bushwick,Brooklyn,"(40.694507, -73.929107)",40.694507,-73.929107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
433,32K549,Bushwick School for Social Justice,Brooklyn,"(40.69697, -73.910791)",40.696970,-73.910791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
434,32K552,Academy of Urban Planning,Brooklyn,"(40.69697, -73.910791)",40.696970,-73.910791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
435,32K554,All City Leadership Secondary School,Brooklyn,"(40.69737, -73.913171)",40.697370,-73.913171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
x.isnull().sum()

dbn                 0
school_name         0
borough             0
location            0
latitude            1
longitude           1
avg_perc Grads    240
trees             239
crashes           239
pc                239
shootings         239
arrests           239
programs          239
bins              239
dtype: int64